In [2]:
# trace
# Converting to Torch Script via Tracing
import torch
import torchvision

# An instance of your model.
model = torchvision.models.resnet18()

# An example input you would normally provide to your model's forward() method.
example = torch.rand(1, 3, 224, 224)

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
traced_script_module = torch.jit.trace(model, example)

In [3]:
# trace 1
output = traced_script_module(torch.ones(1, 3, 224, 224))
output1 = model(torch.ones(1, 3, 224, 224))

output[0, :5],output1[0, :5]

(tensor([ 0.7798, -0.3657,  0.2504,  0.4290, -0.2558], grad_fn=<SliceBackward0>),
 tensor([ 0.7798, -0.3657,  0.2504,  0.4290, -0.2558], grad_fn=<SliceBackward0>))

In [4]:
# .code
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(traced_script_module.code)

('def forward(self,\n'
 '    x: Tensor) -> Tensor:\n'
 '  fc = self.fc\n'
 '  avgpool = self.avgpool\n'
 '  layer4 = self.layer4\n'
 '  layer3 = self.layer3\n'
 '  layer2 = self.layer2\n'
 '  layer1 = self.layer1\n'
 '  maxpool = self.maxpool\n'
 '  relu = self.relu\n'
 '  bn1 = self.bn1\n'
 '  conv1 = self.conv1\n'
 '  _0 = (relu).forward((bn1).forward((conv1).forward(x, ), ), )\n'
 '  _1 = (layer1).forward((maxpool).forward(_0, ), )\n'
 '  _2 = (layer3).forward((layer2).forward(_1, ), )\n'
 '  _3 = (avgpool).forward((layer4).forward(_2, ), )\n'
 '  input = torch.flatten(_3, 1)\n'
 '  return (fc).forward(input, )\n')


In [ ]:

#annotation
# Converting to Torch Script via Annotation
class MyModule(torch.nn.Module):
    def __init__(self, N, M):
        super(MyModule, self).__init__()
        self.weight = torch.nn.Parameter(torch.rand(N, M))

    def forward(self, input):
        if input.sum() > 0:
          output = self.weight.mv(input)
        else:
          output = self.weight + input
        return output

my_module = MyModule(10,20)
sm = torch.jit.script(my_module)

In [ ]:
#file
# Serializing Your Script Module to a File
traced_script_module.save("traced_resnet_model.pt")